In [3]:
import pandas as pd 
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt


In [16]:
test_phase = pd.read_csv('../output/241028_AlgosAlteration/17peps_test_total_phasecuts.csv', index_col=0)
valid_phase = pd.read_csv('../output/241028_AlgosAlteration/17peps_valid_total_phasecuts.csv', index_col=0)
def get_best_eval_string(row):
    return f"dm==\"{row['dm']}\" and phase=={row['phase']} and tree==\"{row['tree']}\" and p=={row['p']}"
cs = ['top','sorted_by','dm', 'tree', 'p', 'phase', 'silhouette', 'mean_purity', 'retention', 'mean_cluster_size', 'n_above', 'n_cluster', 'aggmetric']
valid_phase.apply(get_best_eval_string, axis=1)

0       dm=="TSCS" and phase==0.1 and tree=="tree" and...
1       dm=="TSCS" and phase==0.1 and tree=="tree_aug"...
2       dm=="TSCS" and phase==0.1 and tree=="tree_prun...
3       dm=="TSCS" and phase==0.1 and tree=="tree_prun...
4       dm=="TSCS" and phase==0.1 and tree=="tree_aug_...
                              ...                        
2313    dm=="OSCS" and phase==0.3 and tree=="tree_aug"...
270     dm=="OSCS" and phase==0.3 and tree=="tree_prun...
576     dm=="OSCS" and phase==0.3 and tree=="tree_prun...
1134    dm=="OSCS" and phase==0.3 and tree=="tree_aug_...
2374    dm=="OSCS" and phase==0.3 and tree=="pa_tree" ...
Length: 250, dtype: object

In [87]:
a

,Unnamed: 0,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,silhouette_aggregation,best,dm,phase,tree,p,sa,aggmetric
43,1763.0,0.0488,0.766301,0.3889,4.056738,141.0,10.0,micro,True,TSCS,0.1,tree_aug,40,micro,0.577600
71,1455.0,0.0572,0.660733,0.5880,3.949772,219.0,9.0,micro,True,TSCS,0.2,tree_aug,50,micro,0.624367
75,1473.0,0.0533,0.766657,0.3963,3.912752,149.0,11.0,micro,True,TSCS,0.2,pa_tree,50,micro,0.581479


In [18]:
print('TSCS')
a=test_phase.query('dm=="TSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('mean_cluster_size', ascending=False).head(3)
# display(a)
b=test_phase.query('dm=="TSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head(3)
# display(b)
print('OSCS')
e=test_phase.query('dm=="OSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('mean_cluster_size', ascending=False).head(3)
# display(e)
f=test_phase.query('dm=="OSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head(3)
# display(f)
print('TBCR')
c=test_phase.query('dm=="TBCR"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('mean_cluster_size', ascending=False).head(3)
# display(c)
d=test_phase.query('dm=="TBCR"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head(3)
# display(d)
res=pd.concat([a.assign(sorted_by='mean_cluster_size', top=range(1,4)),
                   b.assign(sorted_by='n_above', top=range(1,4)),
                   e.assign(sorted_by='mean_cluster_size', top=range(1,4)),
                   f.assign(sorted_by='n_above', top=range(1,4)),
                   c.assign(sorted_by='mean_cluster_size', top=range(1,4)),
                   d.assign(sorted_by='n_above', top=range(1,4))])[cs]
display(res)
res.to_excel('../../../writing/VAE_TCR_DRAFT/best_parameters_test_sets.xlsx', index=False)

TSCS
OSCS
TBCR


,top,sorted_by,dm,tree,p,phase,silhouette,mean_purity,retention,mean_cluster_size,n_above,n_cluster,aggmetric
928,1,mean_cluster_size,TSCS,tree_aug_pruned,60,0.50,0.0875,0.908504,0.4974,4.369231,10.0,65.0,0.702952
603,2,mean_cluster_size,TSCS,tree_aug,30,0.15,0.0945,0.911348,0.5499,4.243243,10.0,74.0,0.730624
831,3,mean_cluster_size,TSCS,pa_tree,50,0.50,0.0834,0.918771,0.4886,4.227273,10.0,66.0,0.703686
820,1,n_above,TSCS,tree_aug,50,0.30,0.1096,0.864609,0.6743,3.969072,13.0,97.0,0.769454
708,2,n_above,TSCS,tree_aug,40,0.20,0.0956,0.883301,0.6357,4.172414,12.0,87.0,0.759501
813,3,n_above,TSCS,tree_aug,50,0.25,0.0963,0.870818,0.6427,4.032967,11.0,91.0,0.756759
296,1,mean_cluster_size,OSCS,tree_pruned_aug,50,0.10,0.0543,0.685510,0.4381,4.075949,10.0,158.0,0.561805
1002,2,mean_cluster_size,OSCS,tree_aug,20,0.10,0.0542,0.696111,0.4469,3.864706,8.0,170.0,0.571506
239,3,mean_cluster_size,OSCS,tree_pruned_aug,40,0.10,0.0548,0.761864,0.3544,3.748201,12.0,139.0,0.558132
1088,1,n_above,OSCS,tree,20,0.10,0.0658,0.760067,0.3619,3.500000,13.0,152.0,0.560983


In [21]:
0.5*(a['mean_purity']+a['retention'])

43    0.577600
71    0.624367
75    0.581479
dtype: float64

In [34]:
print('TSCS')
a=valid_phase.query('dm=="TSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('mean_cluster_size', ascending=False).head(3)
# display(a)
b=valid_phase.query('dm=="TSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head(3)
# display(b)
print('OSCS')
e=valid_phase.query('dm=="OSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('mean_cluster_size', ascending=False).head(3)
# display(e)
f=valid_phase.query('dm=="OSCS"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head(3)
# display(f)
print('TBCR')
c=valid_phase.query('dm=="TBCR"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('mean_cluster_size', ascending=False).head(3)
# display(c)
d=valid_phase.query('dm=="TBCR"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head(3)
# display(d)
res=pd.concat([a.assign(sorted_by='mean_cluster_size', top=range(1,4)),
                   b.assign(sorted_by='n_above', top=range(1,4)),
                   e.assign(sorted_by='mean_cluster_size', top=range(1,4)),
                   f.assign(sorted_by='n_above', top=range(1,4)),
                   c.assign(sorted_by='mean_cluster_size', top=range(1,4)),
                   d.assign(sorted_by='n_above', top=range(1,4))])[cs]
res['best_string'] = '('+res.apply(get_best_eval_string, axis =1)+')'
display(res)
res.to_excel('../../../writing/VAE_TCR_DRAFT/best_parameters_valid_sets.xlsx', index=False)

TSCS
OSCS
TBCR


,top,sorted_by,dm,tree,p,phase,silhouette,mean_purity,retention,mean_cluster_size,n_above,n_cluster,aggmetric,best_string
43,1,mean_cluster_size,TSCS,tree_aug,40,0.1,0.0488,0.766301,0.3889,4.056738,10.0,141.0,0.577600,"(dm==""TSCS"" and phase==0.1 and tree==""tree_aug..."
71,2,mean_cluster_size,TSCS,tree_aug,50,0.2,0.0572,0.660733,0.5880,3.949772,9.0,219.0,0.624367,"(dm==""TSCS"" and phase==0.2 and tree==""tree_aug..."
75,3,mean_cluster_size,TSCS,pa_tree,50,0.2,0.0533,0.766657,0.3963,3.912752,11.0,149.0,0.581479,"(dm==""TSCS"" and phase==0.2 and tree==""pa_tree""..."
50,1,n_above,TSCS,tree_aug,40,0.2,0.0610,0.664572,0.6091,3.702479,14.0,242.0,0.636836,"(dm==""TSCS"" and phase==0.2 and tree==""tree_aug..."
65,2,n_above,TSCS,tree_pruned,50,0.1,0.0666,0.848892,0.2916,3.404762,13.0,126.0,0.570246,"(dm==""TSCS"" and phase==0.1 and tree==""tree_pru..."
44,3,n_above,TSCS,tree_pruned,40,0.1,0.0661,0.844152,0.3039,3.465116,13.0,129.0,0.574026,"(dm==""TSCS"" and phase==0.1 and tree==""tree_pru..."
161,1,mean_cluster_size,OSCS,tree_pruned_aug,40,0.1,0.0522,0.758192,0.4099,4.020000,10.0,150.0,0.584046,"(dm==""OSCS"" and phase==0.1 and tree==""tree_pru..."
989,2,mean_cluster_size,OSCS,tree_aug,20,0.1,0.0509,0.711249,0.4623,3.908046,8.0,174.0,0.586774,"(dm==""OSCS"" and phase==0.1 and tree==""tree_aug..."
756,3,mean_cluster_size,OSCS,tree_aug_pruned,50,0.2,0.0455,0.782982,0.3617,3.827338,7.0,139.0,0.572341,"(dm==""OSCS"" and phase==0.2 and tree==""tree_aug..."
1072,1,n_above,OSCS,tree,20,0.1,0.0582,0.776067,0.3732,3.611842,15.0,152.0,0.574634,"(dm==""OSCS"" and phase==0.1 and tree==""tree"" an..."


In [30]:
test_phase['best_string'] = '('+test_phase.apply(get_best_eval_string, axis =1)+')'
best_string = f"({(') or (').join(pd.concat([a,b,c,d, e,f]).apply(get_best_eval_string, axis=1).values)})"
# test_phase.drop_duplicates('aggmetric', keep='last', inplace=True)
wtf=best_string.split(' or ')
for x in wtf:
    print(x, len(test_phase.query(x).drop_duplicates('aggmetric')))

(dm=="TSCS" and phase==0.1 and tree=="tree_aug" and p==40) 1
(dm=="TSCS" and phase==0.2 and tree=="tree_aug" and p==50) 1
(dm=="TSCS" and phase==0.2 and tree=="pa_tree" and p==50) 1
(dm=="TSCS" and phase==0.2 and tree=="tree_aug" and p==40) 1
(dm=="TSCS" and phase==0.1 and tree=="tree_pruned" and p==50) 1
(dm=="TSCS" and phase==0.1 and tree=="tree_pruned" and p==40) 1
(dm=="TBCR" and phase==0.2 and tree=="pa_tree" and p==20) 0
(dm=="TBCR" and phase==0.2 and tree=="tree_aug" and p==30) 1
(dm=="TBCR" and phase==0.2 and tree=="tree_aug" and p==40) 1
(dm=="TBCR" and phase==0.1 and tree=="tree_pruned_aug" and p==30) 1
(dm=="TBCR" and phase==0.1 and tree=="tree_pruned" and p==30) 1
(dm=="TBCR" and phase==0.3 and tree=="tree_aug" and p==40) 1
(dm=="OSCS" and phase==0.1 and tree=="tree_pruned_aug" and p==40) 1
(dm=="OSCS" and phase==0.1 and tree=="tree_aug" and p==20) 1
(dm=="OSCS" and phase==0.2 and tree=="tree_aug_pruned" and p==50) 1
(dm=="OSCS" and phase==0.1 and tree=="tree" and p==20) 1


In [ ]:
0711952675

In [97]:
pd.read_csv('../output/241028_AlgosAlteration/241030_17peps_phases/TSCS_p40_micro_phase020_tree_aug_curve.csv').query('best')


,Unnamed: 0,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,silhouette_aggregation,best,dm,phase,tree,p,sa
1443,1443,0.0651,0.701342,0.4796,3.710526,190.0,9.0,micro,True,TSCS,0.2,tree_aug,40,micro


In [98]:
pd.read_csv('../output/241028_AlgosAlteration/241030_17peps_phases/TBCR_p40_micro_phase020_tree_aug_curve.csv').query('best')

,Unnamed: 0,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,silhouette_aggregation,best,dm,phase,tree,p,sa
3191,3191,0.0368,0.848761,0.234,4.095238,84.0,8.0,micro,True,TBCR,0.2,tree_aug,40,micro


In [94]:
test_phase.query('dm=="TBCR" and phase==0.2 and p==25 and tree=="ap_tree"')

,Unnamed: 0,iteration,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,best,dm,tree,p,sa,phase,aggmetric,silhouette_aggregation,best_string
27,8341.0,60.0,0.049,0.955303,0.2697,3.5,44.0,4.0,True,TBCR,ap_tree,25,micro,0.2,0.612502,NaN,"(dm==""TBCR"" and phase==0.2 and tree==""ap_tree""..."
76,23230.0,60.0,0.049,0.955303,0.2697,3.5,44.0,4.0,True,TBCR,ap_tree,25,macro,0.2,0.612502,NaN,"(dm==""TBCR"" and phase==0.2 and tree==""ap_tree""..."


In [31]:
ntm=pd.concat([test_phase.query(x) for x in wtf]).drop_duplicates('best_string')
print(len(ntm))
ntm.query('retention>.4').sort_values('mean_purity', ascending=False)

17


,Unnamed: 0,iteration,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,best,dm,tree,p,sa,phase,aggmetric,silhouette_aggregation,best_string
120,38474.0,444.0,0.0619,0.928749,0.4396,4.254237,59.0,9.0,True,TBCR,tree_aug,30,micro,0.2,0.684175,NaN,"(dm==""TBCR"" and phase==0.2 and tree==""tree_aug..."
708,83175.0,364.0,0.0956,0.883301,0.6357,4.172414,87.0,12.0,True,TSCS,tree_aug,40,micro,0.2,0.759501,NaN,"(dm==""TSCS"" and phase==0.2 and tree==""tree_aug..."
218,79219.0,151.0,0.0512,0.751702,0.9615,8.854839,62.0,10.0,True,TBCR,tree_aug,40,micro,0.2,0.856601,NaN,"(dm==""TBCR"" and phase==0.2 and tree==""tree_aug..."
1002,NaN,NaN,0.0542,0.696111,0.4469,3.864706,170.0,8.0,True,OSCS,tree_aug,20,micro,0.1,0.571506,micro,"(dm==""OSCS"" and phase==0.1 and tree==""tree_aug..."


In [41]:
xd = ntm.drop(columns=['Unnamed: 0', 'iteration', 'best', 'silhouette_aggregation']).merge(res[['best_string','top','sorted_by']], left_on=['best_string'], right_on=['best_string'])
xd['selection_criterion'] = xd.apply(lambda x: f'Sorted by {x["dm"]} {x["sorted_by"]} from vailid set', axis=1)
xd.to_excel('../../../writing/VAE_TCR_DRAFT/selected_models_on_test_set.xlsx')

In [ ]:
exp

In [80]:
exp_df = pd.read_csv('../data/filtered/240326_nettcr_paired_NOswaps.csv')
top17 = exp_df.query('partition==1').groupby(['peptide']).agg(count=('B3','count')).query('count>=20').index
print(len(top17))
exp17 = exp_df.query('peptide in @top17')
exp17['source_organism'].fillna('missing', inplace=True)
gb=exp17.query('not source_organism=="missing"').groupby(['peptide']).sample(1,random_state=13).groupby(['peptide', 'source_organism']).count()
print(len(gb))
gb

17
15


/var/folders/62/pqfhzdzx689fgftqcgjrgbcw0000gp/T/ipykernel_1556/101140515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp17['source_organism'].fillna('missing', inplace=True)


,,Unnamed: 0,A1,A2,A3,B1,B2,B3,allele,partition,binder,origin,original_peptide,raw_index,reference,input_type
peptide,source_organism,,,,,,,,,,,,,,,
ATDALMTGF,HCV,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
AVFDRKSDAK,EBV,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
CINGVCWTV,HCV,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
ELAGIGILTV,HomoSapiens,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
GILGFVFTL,InfluenzaA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
GLCTLVAML,EBV,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
IVTDFSVIK,EBV,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
KTFPPTEPK,SARS-CoV-2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
LLWNGPMAV,YFV,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [86]:
xd=[x for x in top17 if x not in [x[0] for x in gb.index]]
xd

['KLGGALQAK', 'RAKFKQLL']

In [85]:
pd.read_csv('../data/filtered/231205_nettcr_old_26pep_with_swaps.csv').query('peptide in @xd')

,Unnamed: 0,A1,A2,A3,B1,B2,B3,peptide,allele,origin,binder,partition,original_peptide,original_index
2,2,TSGFNG,NVLDGL,AVGDDKII,DFQATT,SNEGSKA,SARGLDRGTNEQY,KLGGALQAK,HLA-A*03:01,peptide_swapped,0,3,AVFDRKSDAK,3592
3,3,DRGSQS,IYSNGD,AVTPGTYKYI,LGHDT,YNNKEL,ASSPGTSIFVAEQY,KLGGALQAK,HLA-A*03:01,peptide_swapped,0,0,ELAGIGILTV,5933
5,5,DRGSQS,IYSNGD,APNSGGGADGLT,LGHNT,FRNRAP,ASGKLAGVLSEQF,RAKFKQLL,HLA-B*08:01,peptide_swapped,0,2,ELAGIGILTV,1818
8,8,NSASDY,IRSNMDK,AETPSTDSWGKSQ,LNHDA,SQIVND,ASSMQTSGSAYNEQF,KLGGALQAK,HLA-A*03:01,peptide_swapped,0,4,KSKRTPMGF,5434
10,10,SIFNT,LYKAGEL,AGFLYNNNDMR,LGHDT,YNNKEL,ASSHERDRDTGELF,KLGGALQAK,HLA-A*03:01,peptide_swapped,0,3,NLVPMVATV,437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37707,37707,DSVNN,IPSGT,AVTGFNKFY,MRHNA,SNTAGT,ASSDRGLPNEQF,RAKFKQLL,HLA-B*08:01,peptide_swapped,0,2,KLGGALQAK,5144
37711,37711,SSNFYA,MTLNGDE,AFRGSSNTGKLI,SGDLS,YYNGEE,ASSVTDYNEQF,KLGGALQAK,HLA-A*03:01,peptide_swapped,0,1,AVFDRKSDAK,4408
37712,37712,NSAFQY,TYSSGN,AMSLIVNTPLV,SGHTA,FQGNSA,ASSLIGGGKNQPQH,KLGGALQAK,HLA-A*03:01,10x,1,0,KLGGALQAK,4153
37718,37718,NSAFQY,TYSSGN,AMNGARLMFGD,DFQATT,SNEGSKA,SARDVLAGGSDTQYFG,KLGGALQAK,HLA-A*03:01,peptide_swapped,0,3,NLVPMVATV,5763


In [72]:
exp17.query('not source_organism=="missing"')

,Unnamed: 0,peptide,A1,A2,A3,B1,B2,B3,allele,partition,binder,origin,original_peptide,raw_index,reference,source_organism,input_type
0,3,TTDPSFLGRY,ATGYPS,ATKADDK,ALSGNTDKLI,MDHEN,SYDVKM,ASSLAGGIVDEQF,HLA-A*01:01,1,1,VDJdb,TTDPSFLGRY,VDJdb_69875,https://github.com/antigenomics/vdjdb-db/issue...,SARS-CoV-2,tcr_pep
38,260,LLWNGPMAV,NSASQS,VYSSG,VVSGFKTI,MNHEY,SVGEGT,ASSDTDRGIYGYT,HLA-A*02:01,1,1,VDJdb,LLWNGPMAV,VDJdb_6564,28103239,YFV,tcr_pep
49,325,SPRWYFYYL,NYSPAY,IRENEKE,ALDIHTGGFKTI,SGHKS,YYEKEE,ASSLGSEQY,HLA-B*07:02,1,1,VDJdb,SPRWYFYYL,VDJdb_82463,35750048,SARS-CoV-2,tcr_pep
61,421,ELAGIGILTV,TSGFNG,NVLDGL,AVIQDSNYQLI,MNHNS,SASEGT,ASSEAELLSSYNEQF,HLA-A*02:01,1,1,VDJdb,ELAGIGILTV,VDJdb_691,12555663,HomoSapiens,tcr_pep
67,465,GLCTLVAML,NSMFDY,ISSIKDK,AARFDSGYALN,SGDLS,YYNGEE,ASSTYSGQSYGYT,HLA-A*02:01,1,1,VDJdb,GLCTLVAML,VDJdb_65506,28636592,EBV,tcr_pep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9675,58174,KTFPPTEPK,NTAFDY,IRPDVSE,AAPGGRGGTSYGKLT,LNHDA,SQIVND,ASSMRSNQPQH,HLA-A*03:01,1,1,VDJdb,KTFPPTEPK,VDJdb_80879,35750048,SARS-CoV-2,tcr_pep
9687,58234,NLVPMVATV,TISGTDY,GLTSN,ILLGTGANNLF,KGHSH,LQKENI,ASSPKLQGAWAEKLF,HLA-A*02:01,1,1,VDJdb,NLVPMVATV,VDJdb_16624,16237109,CMV,tcr_pep
9703,58302,SPRWYFYYL,SSVPPY,YTSAATLV,AVPLYSSASKII,LNHNV,YYDKDF,ATSRSGTSGVNEQF,HLA-B*07:02,1,1,VDJdb,SPRWYFYYL,VDJdb_80587,35750048,SARS-CoV-2,tcr_pep
9704,58310,SPRWYFYYL,DSASNY,IRSNVGE,AASRGVGGYNKLI,MDHEN,SYDVKM,ASSFLTGPKGDYT,HLA-B*07:02,1,1,VDJdb,SPRWYFYYL,VDJdb_73340,33945786,SARS-CoV-2,tcr_pep


In [47]:
valid_phase.query('dm=="TBCR"').drop_duplicates('aggmetric').query('mean_purity>.65').sort_values('n_above', ascending=False).head()

,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,silhouette_aggregation,best,dm,phase,tree,p,sa,aggmetric
108,0.0510,0.862454,0.3161,3.875000,120.0,17.0,micro,True,TBCR,0.1,tree_pruned_aug,30,micro,0.589277
107,0.0498,0.861658,0.3120,3.701613,124.0,16.0,micro,True,TBCR,0.1,tree_pruned,30,micro,0.586829
141,0.0524,0.714239,0.4915,3.597015,201.0,15.0,micro,True,TBCR,0.3,tree_aug,40,micro,0.602869
164,0.0563,0.754757,0.4296,3.530726,179.0,15.0,micro,True,TBCR,0.3,tree_pruned_aug,50,micro,0.592178
165,0.0437,0.801437,0.3413,3.921875,128.0,13.0,micro,True,TBCR,0.3,tree_aug_pruned,50,micro,0.571368


In [46]:
test_phase.query('dm=="TSCS" and p>=30 and (phase==.2 or phase==.1)  and tree=="tree_aug"')

,iteration,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,best,dm,tree,p,sa,phase,aggmetric
610,331,0.1006,0.883225,0.6287,3.778947,95.0,9.0,True,TSCS,tree_aug,30,micro,0.2,0.819594
694,803,0.0849,0.978723,0.2469,3.000000,47.0,2.0,True,TSCS,tree_aug,40,micro,0.1,0.795768
708,364,0.0956,0.883301,0.6357,4.172414,87.0,12.0,True,TSCS,tree_aug,40,micro,0.2,0.821401
890,1480,0.0829,0.988636,0.2277,2.954545,44.0,1.0,True,TSCS,tree_aug,60,micro,0.1,0.798402


In [38]:
test_phase.query('phase==.1 and dm=="TBCR" and p<=30').query('tree.str.contains("pruned")')

,Unnamed: 0,iteration,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,best,dm,tree,p,sa,phase,aggmetric
9,3397,48,0.0462,0.965833,0.2364,3.375000,40.0,4.0,True,TBCR,tree_pruned,25,micro,0.1,0.783475
107,33764,116,0.0484,0.949074,0.1594,2.527778,36.0,0.0,True,TBCR,tree_pruned,30,micro,0.1,0.751656
109,34166,172,0.0432,0.982759,0.1261,2.482759,29.0,0.0,True,TBCR,tree_aug_pruned,30,micro,0.1,0.768594


In [15]:
test_phase.query('phase==.2 and dm=="TBCR"').query('tree=="tree"')

,Unnamed: 0,iteration,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,n_above,best,dm,tree,p,sa,phase,aggmetric
21,6855,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,25,micro,0.2,0.777039
70,21744,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,25,macro,0.2,0.777039
119,37935,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,30,micro,0.2,0.777039
168,55862,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,30,macro,0.2,0.777039
217,78973,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,40,micro,0.2,0.777039
266,108996,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,40,macro,0.2,0.777039
315,145271,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,50,micro,0.2,0.777039
364,189882,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,50,macro,0.2,0.777039
413,246904,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,60,micro,0.2,0.777039
462,320474,474,0.0615,0.948485,0.2627,2.727273,55.0,1.0,True,TBCR,tree,60,macro,0.2,0.777039


In [22]:
%ls ../output/240924_PeptideRedoClustering | grep 17 | grep valid
topn = pd.read_csv('../output/240924_PeptideRedoClustering/17peps_valid_test_cut_micro_macro_results.csv')
topn['best'] = False
topn.loc[topn.groupby(['silhouette_aggregate', 'clustering','input_type','partition'])['silhouette'].idxmax(), 'best']=True
topn.query('best and silhouette_aggregate=="micro" and partition=="test"')

17peps_valid_test_MM_missingKMeans.csv
17peps_valid_test_agglo_macro_avg_results.csv
17peps_valid_test_agglo_micro_avg_results.csv
17peps_valid_test_cut_micro_macro_results.csv
17peps_valid_test_microMacro_allClustering.csv
Top17_peps_valid_partition.png


,Unnamed: 0,silhouette,mean_purity,retention,mean_cluster_size,n_cluster,silhouette_aggregate,clustering,input_type,partition,best
9982,9982,0.0859,0.680065,0.7190,2.072549,510.0,micro,top1cut,os_cstrp,test,True
12299,12299,0.0970,0.687856,0.7449,2.077799,527.0,micro,top1cut,ts_cstrp,test,True
14609,14609,0.1042,0.645202,0.7646,2.128788,528.0,micro,top1cut,os_notrp,test,True
16972,16972,0.1058,0.670528,0.7408,2.102317,518.0,micro,top1cut,ts_notrp,test,True
21671,21671,0.0464,0.674349,0.7136,2.102204,499.0,micro,top1cut,TBCRalign,test,True
26354,26354,0.0771,0.628686,0.6952,2.142558,477.0,micro,top1cut,tcrdist3,test,True


In [25]:
query=topn.query('best')
query['aggmetric'] = 0.5*(query['mean_purity']+query['retention'])

/var/folders/62/pqfhzdzx689fgftqcgjrgbcw0000gp/T/ipykernel_1556/3658245890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['aggmetric'] = 0.5*(query['mean_purity']+query['retention'])


In [28]:
query.to_excel('../../../writing/VAE_TCR_DRAFT/TopN_selection.xlsx')